In [6]:
import networkx as nx
import numpy as np
import cvxpy as cp
import pandas as pd

Problem setup: 

Paramters: 
- $X \in \mathbb{R}^{n \times n}$ where $X_{i,j}$ represents the amt transit from i to j. 
    - $X_{m,n} = 0$ if we couldn't transit from i to j  
- $T \in \mathbb{R}^{n \times n}$ where $X_{i,j}$ represents the price of i in terms of j. 

Objective: 
- $\min T X^T$

Constraints: 
- $\sum_j X_{ij} - \sum_j X_{ji} = 0 \quad \forall i$  i.e. inflow = outflow 
- $X_{ij} > 0 \quad \forall i,j$

In [106]:
def init(df): 
    # init currency info 
    syms = df.symbol.unique()
    currency_set = set()
    for s in syms: 
        currency_set.add(s.split("/")[0])
        currency_set.add(s.split("/")[1])
    currency_set = list(currency_set)
    currency2index = {}
    for i, s in enumerate(currency_set): 
        currency2index[s] = i
    return currency_set, currency2index

def update_price_mtx(df, currency_set, currency2index): 
    transit_price_mtx = np.ones((len(currency_set), len(currency_set)))
    for i in range(len(df)): 
        ticker = df["symbol"].iloc[i]
        base = ticker.split("/")[0]
        quote = ticker.split("/")[1]
        if base in currency_set and quote in currency_set: 
            base_idx = currency2index[base]
            quote_idx = currency2index[quote]
            transit_price_mtx[base_idx, quote_idx] = df["bid"].iloc[i]
            transit_price_mtx[quote_idx, base_idx] = 1 / df["ask"].iloc[i]
    return transit_price_mtx
    
def get_var_loc(df, currency_set, currency2index): 
    var_loc = np.zeros((len(currency_set), len(currency_set)))
    for s in df.symbol.unique(): 
        base = s.split("/")[0]
        quote = s.split("/")[1]
        base_idx = currency2index[base]
        quote_idx = currency2index[quote]
        var_loc[base_idx, quote_idx] = 1
        var_loc[quote_idx, base_idx] = 1 
    return var_loc

def update_obj(price_mtx, var_loc, currency_set): 
    transit_mtx = np.log(price_mtx)
    # transit_mtx = transit_mtx[var_loc]
    X = cp.Variable((len(currency_set), len(currency_set)))
    obj = cp.Minimize(cp.sum(cp.multiply(transit_mtx, X.T)))
    constraints = [
        X >= 0, 
        X @ np.ones(len(currency_set)) ==  X.T @ np.ones(len(currency_set)), 
        cp.norm(X, "fro") <= 1
    ]
    constraints += [X[i, j] == 0 for i in range(len(currency_set)) for j in range(len(currency_set)) if var_loc[i, j] == 0]
    prob = cp.Problem(obj, constraints) 
    prob.solve()
    print("status:", prob.status)
    print("optimal value", prob.value)
    return X.value


In [46]:
data = pd.read_csv("data.csv")

In [47]:
data.sort_values(by=["timestamp"], inplace=True)

In [107]:
def find_arbitrage(df): 
    currency_set, currency2index = init(df)
    price_mtx = update_price_mtx(df, currency_set, currency2index)
    var_loc = get_var_loc(df, currency_set, currency2index)
    X = update_obj(price_mtx, var_loc, currency_set)

In [108]:
timestamps = data.head(1000).timestamp.unique() 
for t in timestamps: 
    find_arbitrage(data[data["timestamp"] == t])

status: optimal
optimal value -0.026907990861100295
status: optimal
optimal value -0.030902475053949274
status: optimal
optimal value -0.022143441579955736
status: optimal
optimal value -0.016933339230669642
status: optimal
optimal value -0.016880846103179525
status: optimal
optimal value -0.029523149315987318
